In [1]:
import os

os.environ["XRT_TPU_CONFIG"] = "localservice;0;localhost:51011"
print(os.environ)

import gc
gc.enable()

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.distributed import DistributedSampler

import torch_xla
import torch_xla.core.xla_model as xm
import torch_xla.debug.metrics as met
import torch_xla.distributed.parallel_loader as pl
import torch_xla.distributed.xla_multiprocessing as xmp
import torch_xla.utils.serialization as xser
import torch_xla.version as xv

import warnings
warnings.filterwarnings("ignore")

print('PYTORCH:', xv.__torch_gitrev__)
print('XLA:', xv.__xla_gitrev__)

environ({'SHELL': '/bin/bash', 'GLIBCXX_FORCE_NEW': '1', 'CONDA_EXE': '/home/lethanh/miniconda3/bin/conda', '_CE_M': '', 'TMUX': '/tmp/tmux-2001/default,5985,0', 'PWD': '/home/lethanh/workspace/vlsp_viecap4h_gptteam_code/data', 'LOGNAME': 'lethanh', 'XDG_SESSION_TYPE': 'tty', 'CONDA_PREFIX': '/home/lethanh/miniconda3/envs/tpuenv', 'GLIBCPP_FORCE_NEW': '1', 'MOTD_SHOWN': 'pam', 'LD_PRELOAD': '/usr/lib/x86_64-linux-gnu/libtcmalloc.so.4', 'HOME': '/home/lethanh', 'LANG': 'C.UTF-8', 'LS_COLORS': 'rs=0:di=01;34:ln=01;36:mh=00:pi=40;33:so=01;35:do=01;35:bd=40;33;01:cd=40;33;01:or=40;31;01:mi=00:su=37;41:sg=30;43:ca=30;41:tw=30;42:ow=34;42:st=37;44:ex=01;32:*.tar=01;31:*.tgz=01;31:*.arc=01;31:*.arj=01;31:*.taz=01;31:*.lha=01;31:*.lz4=01;31:*.lzh=01;31:*.lzma=01;31:*.tlz=01;31:*.txz=01;31:*.tzo=01;31:*.t7z=01;31:*.zip=01;31:*.z=01;31:*.dz=01;31:*.gz=01;31:*.lrz=01;31:*.lz=01;31:*.lzo=01;31:*.xz=01;31:*.zst=01;31:*.tzst=01;31:*.bz2=01;31:*.bz=01;31:*.tbz=01;31:*.tbz2=01;31:*.tz=01;31:*.deb=01;3

In [2]:
dts = torch.load("./token.pt")

tcmalloc: large alloc 1199423488 bytes == 0x5628f22aa000 @  0x7f4ef7701680 0x7f4ef7722824 0x7f4ef7722b8a 0x7f4e98fa525e 0x7f4e98fa69d2 0x7f4edafc18ed 0x7f4eebbcd8b9 0x7f4eeb8218be 0x5628cf9c6f76 0x5628cf98485f 0x5628cf9d2fa1 0x5628cf94777f 0x5628cf9d1f9f 0x5628cf9d2943 0x5628cf984041 0x5628cfa0999b 0x5628cf9d1a92 0x5628cf9d2943 0x5628cf94777f 0x5628cf9d1a92 0x5628cf9d2754 0x5628cfa60edc 0x5628cfa96083 0x5628cf987699 0x5628cf945b84 0x5628cf9cbdc9 0x5628cfa0c363 0x5628cf9cbdc9 0x5628cfa0c363 0x5628cf9cbdc9 0x5628cf9b71cd


In [3]:
dts.keys()

dict_keys(['en', 'vi'])

In [7]:
type(dts['vi'])

list

In [10]:
lens = []
for i in dts['vi'][:100]:
    lens.append(128 - sum([j == 1 for j in i]))

In [ ]:
!pip  installl matplotli

In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

ModuleNotFoundError: No module named 'matplotlib'

[10,
 78,
 30,
 30,
 18,
 39,
 16,
 10,
 24,
 36,
 20,
 25,
 64,
 50,
 37,
 29,
 34,
 16,
 20,
 27,
 12,
 16,
 26,
 12,
 20,
 25,
 14,
 19,
 13,
 20,
 61,
 14,
 5,
 12,
 24,
 17,
 30,
 16,
 15,
 9,
 21,
 23,
 27,
 19,
 18,
 37,
 25,
 41,
 19,
 16,
 31,
 24,
 5,
 12,
 62,
 27,
 8,
 14,
 23,
 10,
 20,
 22,
 10,
 16,
 7,
 13,
 16,
 15,
 17,
 14,
 42,
 25,
 36,
 18,
 40,
 14,
 19,
 23,
 13,
 23,
 19,
 34,
 55,
 13,
 27,
 23,
 26,
 22,
 15,
 31,
 15,
 28,
 26,
 16,
 12,
 20,
 26,
 11,
 26,
 8]

In [5]:
def collate_fn(batch):
    be = [i[0] for i in batch]
    bv = [i[1] for i in batch]
    return torch.cat(be), torch.tensor(bv)

In [11]:
BATCH_SIZE = 64

dataloader = DataLoader(
    list(zip(dts['en'], dts['vi'])),
    batch_size=BATCH_SIZE,
    collate_fn=collate_fn,
    shuffle=True,
    num_workers=8
)

In [10]:
len(dataloader)

49622

In [13]:
import clip
clip_model, preprocess = clip.load("ViT-B/16", device="cpu")
WRAPPED_MODEL = xmp.MpModelWrapper(clip_model)


In [14]:
device = xm.xla_device()
model = WRAPPED_MODEL.to(device).eval()

2021-12-01 03:13:37.992159: E tensorflow/core/framework/op_kernel.cc:1693] OpKernel ('op: "TpuHandleToProtoKey" device_type: "CPU"') for unknown op: TpuHandleToProtoKey
2021-12-01 03:13:37.992233: E tensorflow/core/framework/op_kernel.cc:1693] OpKernel ('op: "TPURoundRobin" device_type: "CPU"') for unknown op: TPURoundRobin


In [18]:
et, vt = next(iter(dataloader))

In [24]:
from tqdm import tqdm

In [25]:
clip_embedding = []
tgt = []


with torch.no_grad():
    for et, vt in tqdm(dataloader):
        outputs = model.encode_text(et.to(device, dtype=torch.long))
        e_embed = outputs.detach().cpu()
        clip_embedding.append(e_embed)
        tgt.append(vt)


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 49622/49622 [29:13<00:00, 28.29it/s]


In [29]:
clip_embedding = torch.cat(clip_embedding)
tgt = torch.cat(tgt)

tcmalloc: large alloc 6504030208 bytes == 0x5636844d2000 @  0x7fa50354d680 0x7fa50356e824 0x7fa50356eb8a 0x7fa4a33f125e 0x7fa4a33f29d2 0x7fa4e4cab2c3 0x7fa4e44af43f 0x7fa4e4c5cefa 0x7fa4e454e6a1 0x7fa4e454ee95 0x7fa4e4a371a5 0x7fa4e49abaa1 0x7fa4e48323c5 0x7fa4e45508d7 0x7fa4e4af3ab8 0x7fa4e4af3b38 0x7fa4e49abaa1 0x7fa4e4831e25 0x7fa4e605a487 0x7fa4e605aa88 0x7fa4e49abaa1 0x7fa4e4831e25 0x7fa4f5da4118 0x563243ee3f76 0x563243ea185f 0x563243f28f56 0x563243eeea92 0x563243eef754 0x563243f7dedc 0x563243fb3083 0x563243ea4699
tcmalloc: large alloc 3252019200 bytes == 0x563808852000 @  0x7fa50354d680 0x7fa50356e824 0x7fa50356eb8a 0x7fa4a33f125e 0x7fa4a33f29d2 0x7fa4e4cab2c3 0x7fa4e44af43f 0x7fa4e4c5cefa 0x7fa4e454e6a1 0x7fa4e454ee95 0x7fa4e4a371a5 0x7fa4e49abaa1 0x7fa4e48323c5 0x7fa4e45508d7 0x7fa4e4af3ab8 0x7fa4e4af3b38 0x7fa4e49abaa1 0x7fa4e4831e25 0x7fa4e605a487 0x7fa4e605aa88 0x7fa4e49abaa1 0x7fa4e4831e25 0x7fa4f5da4118 0x563243ee3f76 0x563243ea185f 0x563243f28f56 0x563243eeea92 0x563243ee

In [30]:
clip_embedding.shape

torch.Size([3175796, 512])

In [31]:
tgt.shape

torch.Size([3175796, 128])

In [33]:
!rm text_sat_b16.pt

In [36]:
from sklearn.model_selection import train_test_split

In [37]:
X_train, X_test, y_train, y_test = train_test_split(clip_embedding, tgt, test_size=0.05, random_state=42)

tcmalloc: large alloc 6178832384 bytes == 0x56398cb0e000 @  0x7fa50354d680 0x7fa50356e824 0x7fa50356eb8a 0x7fa4a33f125e 0x7fa4a33f29d2 0x7fa4e40c7356 0x7fa4e4529849 0x7fa4e4a341ea 0x7fa4e49ff969 0x7fa4e49b60c7 0x7fa4e485a399 0x7fa4e40ae255 0x7fa4e40b0932 0x7fa4e40b2c9b 0x7fa4e44fb6d6 0x7fa4e45139ac 0x7fa4e4a34daf 0x7fa4e486a6f5 0x7fa4e5ff784e 0x7fa4e5ff806f 0x7fa4e4c60325 0x7fa4f5ffbf82 0x563243ed38c9 0x563243f2559d 0x563243eeea92 0x563243eef943 0x563243e6411a 0x563243eeef9f 0x563243eef943 0x563243e62b84 0x563243f79056
tcmalloc: large alloc 3089416192 bytes == 0x5638cbdec000 @  0x7fa50354d680 0x7fa50356e824 0x7fa50356eb8a 0x7fa4a33f125e 0x7fa4a33f29d2 0x7fa4e40c7356 0x7fa4e4529849 0x7fa4e4a341ea 0x7fa4e49ff969 0x7fa4e49b60c7 0x7fa4e485a399 0x7fa4e40ae255 0x7fa4e40b0932 0x7fa4e40b2c9b 0x7fa4e44fb6d6 0x7fa4e45139ac 0x7fa4e4a34daf 0x7fa4e486a6f5 0x7fa4e5ff784e 0x7fa4e5ff806f 0x7fa4e4c60325 0x7fa4f5ffbf82 0x563243ed38c9 0x563243f2559d 0x563243eeea92 0x563243eef943 0x563243e6411a 0x563243ee

In [38]:
X_test.shape

torch.Size([158790, 512])

In [39]:
X_train.shape

torch.Size([3017006, 512])

In [40]:
y_train.shape

torch.Size([3017006, 128])

In [41]:
y_test.shape

torch.Size([158790, 128])

In [42]:
torch.save({"clip_embedding": X_train, "target": y_train}, "text_sat_b16_train.pt")
torch.save({"clip_embedding": X_test, "target": y_test}, "text_sat_b16_test.pt")

tcmalloc: large alloc 3089416192 bytes == 0x563b1a8dc000 @  0x7fa50354d680 0x7fa50356e824 0x563243e84e96 0x563243ecc456 0x7fa4f6026f3b 0x7fa4e540d385 0x7fa4e54098ea 0x7fa4e540dd99 0x7fa4f6027532 0x7fa4f5c6d8be 0x563243ee3f76 0x563243ea185f 0x563243eeffa1 0x563243e6477f 0x563243eeef9f 0x563243eef943 0x563243e62b84 0x563243eeea92 0x563243eef943 0x563243e6477f 0x563243eeea92 0x563243eef754 0x563243f7dedc 0x563243fb3083 0x563243ea4699 0x563243e62b84 0x563243ee8dc9 0x563243f29363 0x563243ee8dc9 0x563243f29363 0x563243ee8dc9
tcmalloc: large alloc 6178832384 bytes == 0x563b1a8dc000 @  0x7fa50354d680 0x7fa50356e824 0x563243e84e96 0x563243ecc456 0x7fa4f6026f3b 0x7fa4e540d385 0x7fa4e54098ea 0x7fa4e540dd99 0x7fa4f6027532 0x7fa4f5c6d8be 0x563243ee3f76 0x563243ea185f 0x563243eeffa1 0x563243e6477f 0x563243eeef9f 0x563243eef943 0x563243e62b84 0x563243eeea92 0x563243eef943 0x563243e6477f 0x563243eeea92 0x563243eef754 0x563243f7dedc 0x563243fb3083 0x563243ea4699 0x563243e62b84 0x563243ee8dc9 0x563243f2

In [43]:
!ls -llth

total 14G
-rw-rw-r-- 1 lethanh lethanh 466M Dec  1 04:55 text_sat_b16_test.pt
-rw-rw-r-- 1 lethanh lethanh 8.7G Dec  1 04:55 text_sat_b16_train.pt
-rw-rw-r-- 1 lethanh lethanh  17K Dec  1 04:54 inference_tpu.ipynb
-rw-rw-r-- 1 lethanh lethanh  15K Dec  1 03:04 normalize.ipynb
-rw-rw-r-- 1 lethanh lethanh 3.5G Dec  1 03:01 token.pt
-rw-rw-r-- 1 lethanh lethanh 1.5G Oct  5 13:48 train.en-vi.json


In [ ]:
test_preds = []

for i, data in tqdm(enumerate(test_data_loader), total=len(test_data_loader)):
    ids = data["input_ids"]
    mask = data["attention_mask"]
    type_ids = data["token_type_ids"]
    ids = ids.to(device, dtype=torch.long)
    mask = mask.to(device, dtype=torch.long)
    type_ids = type_ids.to(device, dtype=torch.long)
    outputs = model(
        input_ids = ids,
        attention_mask = mask,
        token_type_ids = type_ids
    )
    outputs_np = outputs.cpu().detach().numpy().tolist()
    test_preds.extend(outputs_np)